<a href="https://colab.research.google.com/github/borisevstratov/homeworkHSM/blob/master/nlip_hw1_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import packages and libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# 1. Load the data

In [3]:
# mount google drive folder to access dataframe from google colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# specify location and filename
FolderPath = "/content/drive/My Drive/GSOM/2 semester/Natural Language and Image Recognition/Homework/Homework 1/"
FileName = "dataset_simple.csv"
FullPath = FolderPath + FileName

In [5]:
# initialize pandas dataframe from csv file
DataFrame = pd.read_csv(FullPath)
DataFrame.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,label
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,...,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000
mean,-0.004460,-0.013556,-0.003732,-0.526323,0.005371,-0.008539,0.017156,-0.001695,0.016588,-0.011591,...,0.004245,0.004588,0.000104,-0.004114,-0.015524,0.003792,-0.012826,-0.000154,0.008867,0.493149
std,0.991939,1.004086,1.004643,5.892651,0.995250,1.001142,0.995826,1.000576,1.016175,0.994338,...,1.010292,0.996613,0.984661,1.005568,1.000405,0.992878,0.986676,1.003824,0.992667,0.499978
min,-3.769974,-3.617617,-3.753391,-21.830355,-3.952088,-4.302776,-3.693491,-3.769295,-3.582291,-3.964497,...,-3.543944,-3.702521,-3.645616,-3.757164,-3.998032,-3.844831,-4.314576,-3.912707,-3.852604,0.000000
25%,-0.667193,-0.702984,-0.666734,-4.535838,-0.664072,-0.676919,-0.657419,-0.678497,-0.670545,-0.696125,...,-0.686178,-0.674913,-0.663270,-0.676386,-0.690162,-0.660690,-0.671733,-0.689658,-0.653306,0.000000
50%,0.006990,-0.019359,0.002527,-0.596745,0.008596,-0.014751,0.023373,-0.005377,0.007411,-0.023052,...,-0.002653,0.000651,-0.005459,-0.002117,-0.017218,0.005911,-0.009118,0.024641,-0.002785,0.000000
75%,0.655618,0.667009,0.660033,3.367154,0.672424,0.662462,0.697545,0.662622,0.720524,0.668373,...,0.682821,0.680257,0.668761,0.675716,0.660026,0.680975,0.647070,0.674688,0.677321,1.000000
max,3.629974,4.437563,3.724971,20.021875,3.444207,3.502859,3.925565,4.069058,5.209648,3.871048,...,3.616481,4.456758,3.727838,3.959971,3.520052,4.305980,3.671758,3.555786,3.574397,1.000000


In [12]:
# checking the shape of the data
DataFrame.shape

(9999, 1001)

In [13]:
# checking the sample for imbalance
DataFrame['label'].value_counts(normalize=True)

0    0.506851
1    0.493149
Name: label, dtype: float64

The sample is balanced

# 2. Generating the sample data

In [0]:
# create initial train and test samples
X_train, X_test, y_train, y_test = train_test_split(DataFrame.drop('label', axis = 1), DataFrame['label'], 
                                                    test_size=0.3, random_state=113768, shuffle=True)

In [15]:
# scale the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# re-assign names of rows and columns 
X_train_scaled = pd.DataFrame(X_train_scaled)
X_test_scaled = pd.DataFrame(X_test_scaled)

X_train_scaled.columns = X_train.columns.values
X_train_scaled.index = X_train.index

X_test_scaled.columns = X_test.columns.values
X_test_scaled.index = X_test.index

X_train_scaled.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999
count,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,...,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03,6.999000e+03
mean,1.269008e-17,8.486488e-18,-5.377420e-18,-2.112105e-17,2.268351e-18,2.284214e-18,2.461875e-17,-2.522946e-17,1.465704e-17,-6.567114e-18,...,1.649710e-18,-5.756536e-17,-7.094546e-18,3.048791e-17,1.672116e-17,4.761951e-17,-9.929984e-18,1.068346e-17,1.687780e-17,-9.105130e-18
std,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,...,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00,1.000071e+00
min,-3.794443e+00,-3.559883e+00,-3.757881e+00,-3.638255e+00,-3.972147e+00,-4.275815e+00,-3.704513e+00,-3.750296e+00,-3.560468e+00,-3.395792e+00,...,-3.610808e+00,-3.416185e+00,-3.554031e+00,-3.714590e+00,-3.713280e+00,-3.945987e+00,-3.861551e+00,-4.351920e+00,-3.135034e+00,-3.884088e+00
25%,-6.752867e-01,-6.836349e-01,-6.608566e-01,-6.806223e-01,-6.683760e-01,-6.618225e-01,-6.854063e-01,-6.770352e-01,-6.843066e-01,-6.922372e-01,...,-6.810807e-01,-6.791855e-01,-6.845149e-01,-6.733425e-01,-6.658740e-01,-6.691881e-01,-6.681551e-01,-6.637504e-01,-7.030714e-01,-6.629223e-01
50%,1.740959e-02,-1.825350e-03,4.732626e-03,-8.574115e-03,4.656695e-03,-7.300607e-03,5.304856e-03,-7.286954e-03,-3.712826e-03,-6.731354e-03,...,-1.039153e-03,5.231096e-03,3.879236e-03,-1.379303e-02,-4.747503e-03,-1.099247e-03,4.381364e-03,5.561928e-03,1.891295e-02,-1.363645e-02
75%,6.646423e-01,6.851900e-01,6.599639e-01,6.578250e-01,6.688989e-01,6.672276e-01,6.874024e-01,6.679876e-01,6.872277e-01,6.928306e-01,...,6.659222e-01,6.712706e-01,6.713234e-01,6.810947e-01,6.756952e-01,6.752878e-01,6.800340e-01,6.632122e-01,6.831098e-01,6.755519e-01
max,3.659143e+00,4.418584e+00,3.537421e+00,3.502734e+00,3.441352e+00,3.484128e+00,3.548575e+00,4.055397e+00,5.122034e+00,3.803073e+00,...,4.666567e+00,3.572493e+00,3.529391e+00,3.697507e+00,3.889895e+00,3.322318e+00,4.314648e+00,3.729094e+00,3.541315e+00,3.599152e+00


In [16]:
# let's implement random forest as the base classifier for feature selection.
base_clf = RandomForestClassifier(n_estimators=100)

# setting a minimum threshold of 2*mean
sfm = SelectFromModel(base_clf, threshold="2*mean")
sfm.fit(X_train_scaled, y_train)
n_features = sfm.transform(X_train_scaled).shape[1]

print('Features selected:', n_features)

Features selected: 56


In [0]:
# creating scaled test and train samples with only main features
X_train_scaled_main = sfm.transform(X_train_scaled)
X_test_scaled_main = sfm.transform(X_test_scaled)


# 3. Models testing

In [0]:
# define a simple routine for model tuning
def ModelTuning(ModelName, ModelObject, Parameters, X_train, y_train, X_test, y_test):
    clf = GridSearchCV(ModelObject, Parameters, scoring='roc_auc', n_jobs=-1, cv=3)
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print()
    print('ROC-AUC of', ModelName, 'model on cross-validation:', clf.best_score_)
    print('ROC-AUC of', ModelName, 'model on holdout:', roc_auc_score(y_test, clf.best_estimator_.predict(X_test)))

### K Nearest Neighbours

In [0]:
ParamKNN = {
 'n_neighbors':range(5,15,1),
 'p': [1, 2]
}

In [22]:
ModelTuning('KNN', KNeighborsClassifier(), ParamKNN,
            X_train_scaled_main, y_train, X_test_scaled_main, y_test)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=14, p=2,
           weights='uniform')

ROC-AUC of KNN model on cross-validation: 0.9772535039197823
ROC-AUC of KNN model on holdout: 0.9384753901560624


### Logistic Regression

In [0]:
ParamLR = {
 'penalty': ['l1', 'l2'],
 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
}

In [21]:
ModelTuning('LR', LogisticRegression(), ParamLR,
            X_train_scaled_main, y_train, X_test_scaled_main, y_test)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

ROC-AUC of LR model on cross-validation: 0.9063415481975114
ROC-AUC of LR model on holdout: 0.8112178204615179


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Random Forest

In [0]:
ParamRF = {
 'max_depth': [25, 50, 100, None],
 'min_samples_leaf': [1, 5],
 'n_estimators': [200, 400, 600]
}

In [25]:
ModelTuning('RF', RandomForestClassifier(), ParamRF, 
             X_train_scaled_main, y_train, X_test_scaled_main, y_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

ROC-AUC of RF model on cross-validation: 0.9529278046069887
ROC-AUC of RF model on holdout: 0.8840669601173803


# 4. Conclusion

There were built 3 types of ML models: K Nearest Neighbours, Logistic Regression and Random Forest.

The models were tuned and performed with the following results:

* Roc-AUC of K Nearest Neighbours:
  * Cross-validation 97.72%
  * Holdout 93.84%
* Roc-AUC of Logistic Regression:
  * Cross-validation 90.63%
  * Holdout 81.12%
* Roc-AUC of Random Forest:
  * Cross-validation 95.29%
  * Holdout 88.41%


---


The best results on holdout data were performed by the KNN model on selected features with the following parameters:
```
n_neighbors = 14
p = 2
```


